## Лабораторна робота 5 з ІСППР
### Виконали студенти групи КІ-31мп Шабо О.А. та Сотник Д.C.

## 1. Обрати набір даних з розміченими текстовими документами. 
Можна скористатися набором Internet Movie Database (IMdb), який включає 50 000 текстів рецензій на фільми та відношення до них - позитивна або негативна рецензії (мітки класів).
Задача тоді буде полягати у побудові прогнозу щодо віднесення нової рецензії до одного з класів - позитивна або негативна рецензія.

In [1]:
import re
from sklearn.datasets import fetch_20newsgroups

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score

from collections import Counter

import warnings
warnings.filterwarnings("ignore")


categories = ["comp.graphics", "rec.motorcycles"]

# Завантаження набору даних
data = fetch_20newsgroups(
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

# Тексти для обробки
text_data = data.data
labels = data.target

## 2. Очистити текстові дані: вилучити небажані символи, розмітку та інше (за необхідності).



In [2]:
def clean_text(texts):
    cleaned_texts = []
    for text in texts:
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        text = re.sub(r"\s+", " ", text)
        text = text.lower()
        cleaned_texts.append(text)
    return cleaned_texts


cleaned_text_data = clean_text(text_data)

## 3. Підготувати текстові документи для їх наступної подачі на вхід моделей RNN: 

## 3.1. Створити набір даних pytorch.  Розбити набір на навчальну, валідаційну та тестову підмножини.

In [3]:
# Розподілення даних на підмножини
train_size = int(0.8 * len(cleaned_text_data))
val_size = int(0.1 * len(cleaned_text_data))
test_size = len(cleaned_text_data) - train_size - val_size

train_dataset = cleaned_text_data[:train_size]
val_dataset = cleaned_text_data[train_size : train_size + val_size]
test_dataset = cleaned_text_data[train_size + val_size :]


## 3.2. Знайти унікальні слова (лексеми) в навчальному наборі. 
Для цього можна застосувати клас Counter з пакету collections стандартної бібліотеки Python. 

In [4]:
# Підрахунок слів у навчальному наборі
words = Counter()
for text in train_dataset:
    words.update(text.split())

# Створення словника для перетворення слів у індекси, резервування індексу 0 для падінгу
word_to_index = {word: idx + 1 for idx, word in enumerate(words)}

# Створення словника для перетворення індексів назад у слова
index_to_word = {idx: word for word, idx in word_to_index.items()}

word_to_index


{'hey': 1,
 'what': 2,
 'do': 3,
 'we': 4,
 'look': 5,
 'like': 6,
 'a': 7,
 'parts': 8,
 'store': 9,
 'has': 10,
 'it': 11,
 'ever': 12,
 'occurred': 13,
 'to': 14,
 'you': 15,
 'visit': 16,
 'your': 17,
 'dealer': 18,
 'and': 19,
 'fork': 20,
 'out': 21,
 'the': 22,
 'bucks': 23,
 'for': 24,
 'new': 25,
 'one': 26,
 'are': 27,
 'chances': 28,
 'of': 29,
 'someone': 30,
 'happening': 31,
 'have': 32,
 'part': 33,
 'laying': 34,
 'around': 35,
 'much': 36,
 'less': 37,
 'in': 38,
 'working': 39,
 'condition': 40,
 'sheesh': 41,
 'some': 42,
 'edus': 43,
 'besides': 44,
 'i': 45,
 'only': 46,
 'right': 47,
 'side': 48,
 'inverted': 49,
 'gsxr': 50,
 'its': 51,
 'already': 52,
 'been': 53,
 'converted': 54,
 'into': 55,
 'floor': 56,
 'lamp': 57,
 'know': 58,
 'sounds': 59,
 'suspiciously': 60,
 'no': 61,
 'fault': 62,
 'doesnt': 63,
 'even': 64,
 'was': 65,
 'advertised': 66,
 'as': 67,
 'doinggetting': 68,
 'lawyers': 69,
 'loop': 70,
 'sigh': 71,
 'another': 72,
 'naive': 73,
 'illusi

## 3.3: Відображення унікальних слів на унікальні числа

In [5]:
sample_text = cleaned_text_data[5]

print(sample_text)


 i spoke with the author of macwireframe earlier today the cost is but there are no license royalties his name is eric johnson in sacramento ca phone he doesnt have email very nice guy very knowledgeable about graphics seems like he may have a decent package its an object pascal framework that supposedly has a fairly complete set of geometry creation classes im going to check it out and see if its got what i need for my cad package i also found another package d graphic tools by micro system options in seattle the number is also no email the package is strong at ray tracing im not too sure about its geometry creation tools i also need to look into this package some more i also spoke with the author mark owens another nice guy that seems to know his business the price is no royalties


In [6]:
def encode_text(text, word_to_index):
    # Розділення тексту на слова та кодування за допомогою відповідних індексів
    return [word_to_index[word] for word in text.split() if word in word_to_index]


# Приклад використання
encoded_text = encode_text(sample_text, word_to_index)
print(encoded_text)


[45, 259, 191, 22, 260, 29, 261, 262, 263, 22, 264, 108, 187, 265, 27, 61, 266, 267, 268, 269, 108, 270, 271, 38, 272, 273, 274, 275, 63, 32, 105, 276, 184, 277, 276, 278, 223, 100, 279, 6, 275, 280, 32, 7, 281, 282, 51, 87, 283, 284, 285, 92, 286, 10, 7, 287, 288, 289, 29, 290, 291, 292, 293, 294, 14, 295, 11, 21, 19, 296, 139, 51, 232, 2, 45, 297, 24, 110, 298, 282, 45, 202, 299, 72, 282, 300, 301, 302, 161, 303, 304, 305, 38, 306, 22, 307, 108, 202, 61, 105, 22, 282, 108, 308, 90, 309, 310, 293, 311, 312, 313, 223, 51, 290, 291, 302, 45, 202, 297, 14, 5, 55, 95, 282, 42, 238, 45, 202, 259, 191, 22, 260, 314, 315, 72, 184, 277, 92, 279, 14, 58, 268, 316, 22, 317, 108, 61, 267]


## 3.4. Створити вкладення (embedding) ознак, направлене на зменшення розмірності простору ознак (вектору слів).

In [7]:
# Параметри
vocab_size = len(word_to_index) + 1  # Розмір словника плюс один для невідомого слова
embedding_dim = 50  # Розмір виміру вектора вкладень

# Визначення моделі
class TextEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(TextEmbedder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

    def forward(self, x):
        return self.embedding(x)


# Створення екземпляра моделі
model = TextEmbedder(vocab_size, embedding_dim)

# Приклад вхідних даних (передбачається, що encoded_text є вже закодованим текстом)
input_indices = torch.tensor(encoded_text)

# Отримання вкладень для вхідних даних
embeddings = model(input_indices)
print(embeddings)


tensor([[ 0.7792, -1.5540,  0.8456,  ..., -0.4794, -0.8362,  0.3841],
        [-1.1745,  0.7534, -0.2035,  ...,  1.2625,  1.0855, -0.8247],
        [-0.7997,  1.8817, -0.8237,  ..., -0.5258, -0.0156, -2.2362],
        ...,
        [-0.2872, -1.4426,  0.6887,  ..., -1.6810, -1.4018,  0.6076],
        [ 1.6478,  0.4133, -0.9987,  ...,  0.6622, -0.5225,  0.4312],
        [ 1.5270, -0.5407, -0.7717,  ..., -0.2328, -1.7945, -1.6725]],
       grad_fn=<EmbeddingBackward0>)


## 3.5. Розділити навчальний набір даних на міні-пакети, які будуть слугувати входом наступних моделей RNN.

In [8]:
from torch.nn.utils.rnn import pad_sequence


class NewsGroupDataset(Dataset):
    def __init__(self, texts, labels, word_to_index, max_len=None):
        self.texts = [self.encode_text(text, word_to_index) for text in texts]
        self.labels = labels
        self.max_len = max_len

    def encode_text(self, text, word_to_index):
        return [
            word_to_index.get(word, 0) for word in text.split()
        ]  # 0 для невідомих слів

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        if self.max_len:
            text = (
                text[: self.max_len]
                if len(text) > self.max_len
                else text + [0] * (self.max_len - len(text))
            )
        return torch.tensor(text), self.labels[idx]


# При створенні DataLoader використовуйте pad_sequence для динамічного доповнення
def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(
        [torch.tensor(text) for text in texts], batch_first=True, padding_value=0
    )
    return texts_padded, torch.tensor(labels)


dataset = NewsGroupDataset(cleaned_text_data, labels, word_to_index, max_len=500)

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

# Тепер при створенні DataLoader передайте collate_fn
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
)
val_loader = DataLoader(
    train_dataset,
    batch_size=32,
)
test_loader = DataLoader(
    train_dataset,
    batch_size=32,
)


## 4. Побудувати базову глибоку модель RNN, яка включає наступні шари:
   - Embedding,  задати значення input_dim та output_dim,   
   - один або кілька шарів SimpleRNN, (в разі двох шарів в першому з них задати return_sequences=True),
   - останній вихідний шар повнозв'язний Dense з одним нейроном.

Навчити цю модель на 10 - 50 епохах, використовуючи:
- функцію втрат BinaryCrossentropy,  попередньо в останньому шарі задати activation='sigmoid',
- оптимізатор Adam.

Оцінити якість моделі на навчальному та тестовому наборах за показником accuracy.

In [9]:
from pytorch_lightning.callbacks import RichProgressBar


class RNNModel(pl.LightningModule):
    def __init__(self, rnn, input_dim, output_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, output_dim)
        self.rnn = rnn
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])  # Use the last timestep
        x = self.sigmoid(x)
        x = x.squeeze()  # Squeeze the output to remove the extra dimension
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.BCELoss()(y_hat, y.float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def validation_step(self, batch, batch_idx):
        x, y = batch

        y_hat = self.forward(x)
        loss = nn.BCELoss()(y_hat, y.float())
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        y_hat_label = (y_hat > 0.5).float()
        acc = accuracy_score(y.cpu(), y_hat_label.cpu())
        self.log("test_acc", acc)

In [11]:
input_dim = len(word_to_index) + 1

output_dim = 100
hidden_dim = 50

rnn = nn.RNN(output_dim, hidden_dim, num_layers=2, batch_first=True)

model = RNNModel(rnn, input_dim=vocab_size, output_dim=100, hidden_dim=50)

trainer = Trainer(max_epochs=50, enable_checkpointing=False, callbacks=[RichProgressBar()])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ RNN       │ 12.7 K │
│ 2 │ fc        │ Linear    │     51 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.8 M                                                                                                
Total estimated model params size (MB): 7

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5206349492073059     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.5206349492073059}]

## 5. Побудувати кілька альтернативних глибоких моделей RNN на основі базової, в яких замість SimpleRNN використати:
   - LSTM, 
   - GRU.

In [12]:
rnn = nn.LSTM(output_dim, hidden_dim, num_layers=2, batch_first=True)

model = RNNModel(rnn, input_dim=vocab_size, output_dim=100, hidden_dim=50)

trainer = Trainer(max_epochs=50, enable_checkpointing=False, callbacks=[RichProgressBar()])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ LSTM      │ 50.8 K │
│ 2 │ fc        │ Linear    │     51 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.9 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.9 M                                                                                                
Total estimated model params size (MB): 7

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9892063736915588     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9892063736915588}]

In [13]:
rnn = nn.GRU(output_dim, hidden_dim, num_layers=2, batch_first=True)
model = RNNModel(rnn, input_dim=vocab_size, output_dim=100, hidden_dim=50)

trainer = Trainer(max_epochs=50, enable_checkpointing=False, callbacks=[RichProgressBar()])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ GRU       │ 38.1 K │
│ 2 │ fc        │ Linear    │     51 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.9 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.9 M                                                                                                
Total estimated model params size (MB): 7

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9911110997200012     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9911110997200012}]

## 6. Побудувати альтернативну глибоку модель RNN, в якій    
   - помістити шар LSTM (GRU) всередину обгортки Bidirectional, таким чином, що рекурентні шари будуть проходити по вхідним послідовностям в обох напрямках - від початку до кінця і в зворотному напрямку: з кінця до початку.

In [14]:
import torch.nn as nn
import pytorch_lightning as pl


class BidirectionalRNNModel(pl.LightningModule):
    def __init__(self, rnn, input_dim, output_dim, hidden_dim):
        super(BidirectionalRNNModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, output_dim)
        self.rnn = rnn
        self.fc = nn.Linear(
            2 * hidden_dim, 1
        )  # Multiply hidden_dim by 2 because of bidirectionality
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        # Use the concatenated last hidden states from both directions
        x = self.fc(x[:, -1, :])
        x = self.sigmoid(x)
        x = x.squeeze()  # Squeeze the output to remove the extra dimension
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.BCELoss()(y_hat, y.float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.BCELoss()(y_hat, y.float())
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        y_hat_label = (y_hat > 0.5).float()
        acc = accuracy_score(
            y.cpu(), y_hat_label.cpu()
        )  # Make sure to import accuracy_score from an appropriate module
        self.log("test_acc", acc)


In [15]:
rnn = nn.LSTM(output_dim, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)

model = BidirectionalRNNModel(rnn, input_dim=vocab_size, output_dim=100, hidden_dim=50)

trainer = Trainer(max_epochs=50, enable_checkpointing=False, callbacks=[RichProgressBar()])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ LSTM      │  121 K │
│ 2 │ fc        │ Linear    │    101 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.9 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.9 M                                                                                                
Total estimated model params size (MB): 7

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9911110997200012     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9911110997200012}]

In [16]:
rnn = nn.GRU(output_dim, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
model = BidirectionalRNNModel(rnn, input_dim=vocab_size, output_dim=100, hidden_dim=50)

trainer = Trainer(max_epochs=50, enable_checkpointing=False, callbacks=[RichProgressBar()])
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ GRU       │ 91.2 K │
│ 2 │ fc        │ Linear    │    101 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.9 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.9 M                                                                                                
Total estimated model params size (MB): 7

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9911110997200012     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9911110997200012}]

## 7. Оцінити якість всіх побудованих моделей.  
Зробити висновки. Обрати найкращу модель.

| Model | Direction | Accuracy |
|-----|----|--------|
| RNN | Single | 0.5206349492073059 |
| LSTM | Single | 0.9892063736915588 |
| GRU | Single | 0.9911110997200012 |
| LSTM | Bi | 0.9911110997200012 |
| GRU | Bi | 0.9911110997200012 |



## 8. Задати додатковий параметр max_seq_length - довжину вхідної послідовності на етапі знаходження унікальних слів (лексем) в навчальному наборі. 
Поекспериментувати з різними значеннями цього параметру (з короткими та довгими послідовностями).
Порівняти різні моделі RNN з різними довжинами вхідних послідовностей.
Зрозуміти можливості різних моделей RNN по обробці довгих послідовностей. 


In [17]:

class Experiment:
    def __init__(self, max_lens, model_types):
        self.max_lens = max_lens
        self.model_types = model_types

    def run(self):
        results = {}
        for max_len in self.max_lens:
            dataset = NewsGroupDataset(
                cleaned_text_data, labels, word_to_index, max_len=max_len
            )
            train_size = int(0.8 * len(dataset))
            val_size = int(0.1 * len(dataset))
            test_size = len(dataset) - train_size - val_size
            train_dataset, val_dataset, test_dataset = random_split(
                dataset, [train_size, val_size, test_size]
            )

            train_loader = DataLoader(
                train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn
            )
            val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)
            test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

            print(len(train_loader))
            print(len(val_loader))

            for rnn in self.model_types:
                print(f"Training {type(rnn)} with max_len={max_len}")
                model = RNNModel(
                    rnn, input_dim=len(word_to_index) + 1, output_dim=100, hidden_dim=50
                )
                trainer = Trainer(max_epochs=20, callbacks=[RichProgressBar()])
                trainer.fit(model, train_loader, val_loader)
                result = trainer.test(model, test_loader)
                print(result)
                results[(rnn, max_len)] = result

        return results


rnn = nn.RNN(output_dim, hidden_dim, num_layers=2, batch_first=True)
lstm = nn.LSTM(output_dim, hidden_dim, num_layers=2, batch_first=True)
gru = nn.GRU(output_dim, hidden_dim, num_layers=2, batch_first=True)

experiment = Experiment(max_lens=[250, 500, 1000], model_types=[rnn, lstm, gru])
results = experiment.run()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


50
7
Training <class 'torch.nn.modules.rnn.RNN'> with max_len=250


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type      ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ embedding │ Embedding │  1.8 M │
│ 1 │ rnn       │ RNN       │ 12.7 K │
│ 2 │ fc        │ Linear    │     51 │
│ 3 │ sigmoid   │ Sigmoid   │      0 │
└───┴───────────┴───────────┴────────┘

Trainable params: 1.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.8 M                                                                                                
Total estimated model params size (MB): 7

Output()

...

`Trainer.fit` stopped: `max_epochs=20` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8131313323974609     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.8131313323974609}]


In [18]:
for (model, seq_len), accuracies in results.items():
    model_name = type(model).__name__
    accuracy = accuracies[0]['test_acc']
    print(f"Model: {model_name}, Sequence Length: {seq_len}, Test Accuracy: {accuracy:.4f}")

Model: RNN, Sequence Length: 250, Test Accuracy: 0.4697
Model: LSTM, Sequence Length: 250, Test Accuracy: 0.8788
Model: GRU, Sequence Length: 250, Test Accuracy: 0.8990
Model: RNN, Sequence Length: 500, Test Accuracy: 0.4949
Model: LSTM, Sequence Length: 500, Test Accuracy: 0.5101
Model: GRU, Sequence Length: 500, Test Accuracy: 0.8283
Model: RNN, Sequence Length: 1000, Test Accuracy: 0.5303
Model: LSTM, Sequence Length: 1000, Test Accuracy: 0.4697
Model: GRU, Sequence Length: 1000, Test Accuracy: 0.8131
